In [6]:
import numpy as np

# 1️⃣ Tạo ma trận A (5x4)
A = np.array([
    [5, 4, 0, 1],
    [4, 0, 0, 1],
    [1, 1, 0, 5],
    [0, 0, 5, 4],
    [0, 1, 5, 4]
])

# 2️⃣ Thực hiện SVD đầy đủ
U, S, Vt = np.linalg.svd(A, full_matrices=False)

# 3️⃣ Chọn k = 2 (rút gọn SVD)
k = 2
U_r = U[:, :k]  # Lấy 2 cột đầu của U
S_r = np.diag(S[:k])  # Chuyển S thành ma trận đường chéo (2x2)
V_r_T = Vt[:k, :]  # Lấy 2 hàng đầu của V^T

# 4️⃣ In kết quả
print("Ma trận U_r (Người dùng):\n", np.round(U_r, 2))
print("\nMa trận Σ_r (Giá trị kỳ dị):\n", np.round(S_r, 2))
print("\nMa trận V_r^T (Sản phẩm):\n", np.round(V_r_T, 2))

# 5️⃣ Người dùng mới với đánh giá r = [5, 0, 3, 4]
r = np.array([[5, 0, 3, 4]])

# 6️⃣ Tính U_new
S_r_inv = np.linalg.inv(S_r)  # Tính ma trận nghịch đảo của Σ_r
U_new = r @ V_r_T.T @ S_r_inv  # Công thức tính U_new
print("\nMa trận U mới (Người dùng):\n", np.round(U_new, 2))

# 7️⃣ Tính cosine similarity
cosine_similarities = []
for i, U_i in enumerate(U_r):
    cos_alpha = np.dot(U_new, U_i).item() / (np.linalg.norm(U_new) * np.linalg.norm(U_i))

    cosine_similarities.append((i + 1, round(cos_alpha, 2)))  # Làm tròn 2 chữ số

# 8️⃣ In kết quả
print("\nCosine Similarity với người dùng khác:")
for user, cos_sim in cosine_similarities:
    print(f"Người dùng {user}: {cos_sim}")


Ma trận U_r (Người dùng):
 [[-0.28 -0.79]
 [-0.17 -0.44]
 [-0.41 -0.12]
 [-0.59  0.32]
 [-0.61  0.26]]

Ma trận Σ_r (Giá trị kỳ dị):
 [[10.03  0.  ]
 [ 0.    7.19]]

Ma trận V_r^T (Sản phẩm):
 [[-0.25 -0.22 -0.6  -0.73]
 [-0.81 -0.42  0.41  0.07]]

Ma trận U mới (Người dùng):
 [[-0.6  -0.36]]

Cosine Similarity với người dùng khác:
Người dùng 1: 0.77
Người dùng 2: 0.79
Người dùng 3: 0.97
Người dùng 4: 0.51
Người dùng 5: 0.59


In [ ]:
import numpy as np
import pandas as pd
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

# Load dữ liệu
file_path = '20191226-reviews.csv'
df = pd.read_csv(file_path)

# Chỉ giữ lại các cột cần thiết
ratings = df[['name', 'asin', 'rating']].copy()

# Chuyển rating về kiểu số và loại bỏ giá trị NaN
ratings.loc[:, 'rating'] = pd.to_numeric(ratings['rating'], errors='coerce')
ratings = ratings.dropna(subset=['rating'])

# Tính trung bình rating của mỗi người dùng trên mỗi sản phẩm
ratings = ratings.groupby(['name', 'asin'], as_index=False).agg({'rating': 'mean'})

# Lọc ra những người dùng đã đánh giá ít nhất 3 sản phẩm
user_counts = ratings['name'].value_counts()
valid_users = user_counts[user_counts >= 3].index
ratings = ratings[ratings['name'].isin(valid_users)]

# Chuyển dữ liệu thành ma trận user-item
rating_matrix = ratings.pivot(index='name', columns='asin', values='rating').fillna(0)

# Áp dụng SVD với k = 10 
k = min(50, rating_matrix.shape[1])
svd = TruncatedSVD(n_components=k)
U_r = svd.fit_transform(rating_matrix)  # Ma trận U_r
Sigma_r = np.diag(svd.singular_values_)  # Ma trận Σ_r
V_r_T = svd.components_  # Ma trận V_r^T

# Chọn một người dùng bất kỳ trong dataset làm người dùng mới
test_user_index = 20  
r = rating_matrix.iloc[test_user_index].values.reshape(1, -1)

# Tính toán vector U_new cho người dùng mới
Sigma_r_inv = np.linalg.pinv(Sigma_r)  # Ma trận nghịch đảo của Σ_r
U_new = np.dot(np.dot(r, V_r_T.T), Sigma_r_inv)  # U_new = r * Σ_r^(-1) * V_r

# Tính cosine similarity giữa U_new và tất cả người dùng khác
similarities = cosine_similarity(U_new, U_r)[0]

# Lấy 10 người dùng có hệ số cosine cao nhất 
similar_users = np.argsort(similarities)[::-1][0:10]


similar_user_ids = rating_matrix.iloc[similar_users].index
print(f"Size của rating_matrix: {rating_matrix.shape}")
print(f"Size của U_r: {U_r.shape}")
print(f"Size của Sigma_r: {Sigma_r.shape}")
print(f"Size của V_r_T: {V_r_T.shape}")
print(f"Size của r (test user): {r.shape}")
print(f"Size của Sigma_r_inv: {Sigma_r_inv.shape}")
print(f"Size của U_new: {U_new.shape}")


# In danh sách người dùng tương đồng
print("10 người dùng gần nhất với User mới:")
for i, user in enumerate(similar_user_ids):
    print(f"{i+1}. {user} - Cosine Similarity: {similarities[similar_users[i]]:.4f}")

# Lấy danh sách sản phẩm mà người dùng mới đã đánh giá
test_user_rated_products = set(rating_matrix.iloc[test_user_index][rating_matrix.iloc[test_user_index] > 0].index)

# Tìm 10 sản phẩm cao nhất từ từng người dùng tương đồng nhưng chưa được test_user đánh giá và có rating > 3
recommended_products = []
seen_products = set()

for user in similar_user_ids:
    user_top_products = ratings[(ratings['name'] == user) & (ratings['rating'] > 3)].sort_values(by='rating', ascending=False)
    for _, row in user_top_products.iterrows():
        if row['asin'] not in seen_products and row['asin'] not in test_user_rated_products:
            recommended_products.append(row)
            seen_products.add(row['asin'])
        if len(recommended_products) >= 10:
            break
    if len(recommended_products) >= 10:
        break

# Chuyển danh sách sản phẩm thành DataFrame để hiển thị
recommended_products_df = pd.DataFrame(recommended_products)



print("\n10 sản phẩm được gợi ý:")
print(recommended_products_df[['asin', 'rating']])

# Lấy danh sách sản phẩm mà người dùng mới đã đánh giá
test_user_rated_products = set(rating_matrix.iloc[test_user_index][rating_matrix.iloc[test_user_index] > 0].index)

# Lấy danh sách sản phẩm gợi ý
recommended_products_list = set(recommended_products_df['asin'])

# Đếm số lượng sản phẩm gợi ý trùng với sản phẩm đã đánh giá
overlap = recommended_products_list.intersection(test_user_rated_products)
print(f"Số sản phẩm gợi ý trùng với sản phẩm đã đánh giá: {len(overlap)} / {len(recommended_products_list)}")
print("Các sản phẩm trùng:", overlap)


Size của rating_matrix: (1640, 639)
Size của U_r: (1640, 50)
Size của Sigma_r: (50, 50)
Size của V_r_T: (50, 639)
Size của r (test user): (1, 639)
Size của Sigma_r_inv: (50, 50)
Size của U_new: (1, 50)
10 người dùng gần nhất với User mới:
1. Abby - Cosine Similarity: 0.9834
2. Jodi - Cosine Similarity: 0.8441
3. Homer - Cosine Similarity: 0.8419
4. ben - Cosine Similarity: 0.8265
5. nathan - Cosine Similarity: 0.8265
6. DW - Cosine Similarity: 0.8215
7. Wilson - Cosine Similarity: 0.8184
8. tim - Cosine Similarity: 0.8167
9. Gloria - Cosine Similarity: 0.8104
10. Tommy - Cosine Similarity: 0.7970

10 sản phẩm được gợi ý:
             asin  rating
23953  B01NB1KG8U     4.0
19733  B0134TVNSQ     5.0
19734  B01BG1MW5C     5.0
57877  B078YXQ38Z     5.0
57875  B019S654DO     4.0
50195  B07R57VRJ7     5.0
60325  B00WF988BW     5.0
60326  B01N9TM7KG     5.0
60329  B071VGLCRX     4.0
60330  B07VYP6VLS     4.0
Số sản phẩm gợi ý trùng với sản phẩm đã đánh giá: 0 / 10
Các sản phẩm trùng: set()


In [70]:
import numpy as np
import pandas as pd
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
import random

# Load dữ liệu
file_path = '20191226-reviews.csv'
df = pd.read_csv(file_path)

# Chỉ giữ lại các cột cần thiết
ratings = df[['name', 'asin', 'rating']].copy()

# Chuyển rating về kiểu số và loại bỏ giá trị NaN
ratings.loc[:, 'rating'] = pd.to_numeric(ratings['rating'], errors='coerce')
ratings = ratings.dropna(subset=['rating'])

# Tính trung bình rating của mỗi người dùng trên mỗi sản phẩm
ratings = ratings.groupby(['name', 'asin'], as_index=False).agg({'rating': 'mean'})

# Lọc ra những người dùng đã đánh giá ít nhất 3 sản phẩm
user_counts = ratings['name'].value_counts()
valid_users = user_counts[user_counts >= 3].index
ratings = ratings[ratings['name'].isin(valid_users)]

# Chuyển dữ liệu thành ma trận user-item
rating_matrix = ratings.pivot(index='name', columns='asin', values='rating').fillna(0)

# Thêm người dùng mới với 5 sản phẩm ngẫu nhiên và rating ngẫu nhiên từ 1-5
new_user_name = 'Huey'
all_products = rating_matrix.columns.tolist()
new_user_products = random.sample(all_products, 5)
new_user_ratings = [random.randint(1, 5) for _ in range(5)]

# Tạo DataFrame cho người dùng mới
new_user_df = pd.DataFrame({'name': [new_user_name] * 5, 'asin': new_user_products, 'rating': new_user_ratings})
ratings = pd.concat([ratings, new_user_df], ignore_index=True)

# Cập nhật ma trận user-item
rating_matrix = ratings.pivot(index='name', columns='asin', values='rating').fillna(0)

# Áp dụng SVD với k = 10 (hoặc nhỏ hơn nếu số sản phẩm ít)
k = min(50, rating_matrix.shape[1])
svd = TruncatedSVD(n_components=k)
U_r = svd.fit_transform(rating_matrix)  # Ma trận U_r
Sigma_r = np.diag(svd.singular_values_)  # Ma trận Σ_r
V_r_T = svd.components_  # Ma trận V_r^T

# Lấy dữ liệu của người dùng mới
r = rating_matrix.loc[new_user_name].values.reshape(1, -1)

# Tính toán vector U_new cho người dùng mới
Sigma_r_inv = np.linalg.pinv(Sigma_r)  # Ma trận nghịch đảo của Σ_r
U_new = np.dot(np.dot(r, V_r_T.T), Sigma_r_inv)  # U_new = r * Σ_r^(-1) * V_r

# Tính cosine similarity giữa U_new và tất cả người dùng khác
similarities = cosine_similarity(U_new, U_r)[0]

# Lấy 10 người dùng có hệ số cosine cao nhất (bỏ chính mình)
similar_users = np.argsort(similarities)[::-1][1:11]
similar_user_ids = rating_matrix.iloc[similar_users].index

print("10 người dùng gần nhất với người dùng Huey:")
for i, user in enumerate(similar_user_ids):
    print(f"{i+1}. {user} - Cosine Similarity: {similarities[similar_users[i]]:.4f}")

# Lấy danh sách sản phẩm mà người dùng mới đã đánh giá
test_user_rated_products = set(new_user_products)

# Tìm 10 sản phẩm cao nhất từ từng người dùng tương đồng nhưng chưa được test_user đánh giá
recommended_products = []
seen_products = set()

for user in similar_user_ids:
    user_top_products = ratings[(ratings['name'] == user) & (ratings['rating'] > 3)].sort_values(by='rating', ascending=False)
    for _, row in user_top_products.iterrows():
        if row['asin'] not in seen_products and row['asin'] not in test_user_rated_products:
            recommended_products.append(row)
            seen_products.add(row['asin'])
        if len(recommended_products) >= 10:
            break
    if len(recommended_products) >= 10:
        break

# Chuyển danh sách sản phẩm thành DataFrame để hiển thị
recommended_products_df = pd.DataFrame(recommended_products)

# In danh sách 10 sản phẩm gợi ý
print("\n10 sản phẩm được gợi ý:")
print(recommended_products_df[['asin', 'rating']])

# Kiểm tra sự trùng lặp với sản phẩm đã đánh giá
recommended_products_list = set(recommended_products_df['asin'])
overlap = recommended_products_list.intersection(test_user_rated_products)
print(f"Số sản phẩm gợi ý trùng với sản phẩm đã đánh giá: {len(overlap)} / {len(recommended_products_list)}")
print("Các sản phẩm trùng:", overlap)

10 người dùng gần nhất với người dùng Huey:
1. Louie - Cosine Similarity: 0.9261
2. Rebekah - Cosine Similarity: 0.9232
3. Kristi - Cosine Similarity: 0.9203
4. shannon - Cosine Similarity: 0.8987
5. #NAME? - Cosine Similarity: 0.8850
6. LD - Cosine Similarity: 0.8737
7. BC - Cosine Similarity: 0.8548
8. Whitney - Cosine Similarity: 0.8323
9. B - Cosine Similarity: 0.7466
10. J.E. - Cosine Similarity: 0.7005

10 sản phẩm được gợi ý:
             asin  rating
6454   B00NKR9MJA     5.0
8192   B01JAWWVXW     5.0
8194   B07ND4ZN2X     5.0
8191   B00JYR6GGM     4.0
6147   B072N3GKSM     4.0
11098  B00O15MWOM     5.0
11100  B07W14HFQP     5.0
0      B01M01YX15     5.0
2      B0788F8DKC     5.0
6183   B00JEHJMG8     5.0
Số sản phẩm gợi ý trùng với sản phẩm đã đánh giá: 0 / 10
Các sản phẩm trùng: set()
